In [ ]:
import numpy as np
import pandas as pd
import talib as ta
import tushare as ts
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
mpl.rcParams['axes.unicode_minus']=False #确保‘-’号显示正常
mpl.rcParams['font.sans-serif'] = ['SimHei'] #确保中文显示正常

stock_index = ts.get_k_data('hs300', '2016-01-01', '2017-07-01')
stock_index['date'] = pd.to_datetime(stock_index['date'])
stock_index.set_index('date', inplace = True)

stock_index['upper'], stock_index['middle'], stock_index['lower'] = ta.BBANDS(np.asarray(stock_index['close']), timeperiod=20, nbdevup=2, nbdevdn=2, matype=0) #计算布林带，matype指定均线类型

fig = plt.figure(figsize = (10,6))
plt.plot(stock_index['close'])
plt.plot(stock_index['upper'], linestyle='--')
plt.plot(stock_index['middle'], linestyle='--')
plt.plot(stock_index['lower'], linestyle='--')
plt.title('沪深300 布林线图')
plt.legend()

stock_index['yes_close'] = stock_index['close'].shift(1) #计算昨日数据
stock_index['yes_lower'] = stock_index['lower'].shift(1)
stock_index['yes_upper'] = stock_index['upper'].shift(1)
stock_index['daybeforeyes_close'] = stock_index['close'].shift(2) #计算前天数据
stock_index['daybeforeyes_lower'] = stock_index['lower'].shift(2)
stock_index['daybeforeyes_upper'] = stock_index['upper'].shift(2)

stock_index['signal'] = np.where(np.logical_and(stock_index['daybeforeyes_close'] < stock_index['daybeforeyes_lower'], stock_index['yes_close'] > stock_index['yes_lower']), 1, 0)
stock_index['signal'] = np.where(np.logical_and(stock_index['daybeforeyes_close'] > stock_index['daybeforeyes_upper'], stock_index['yes_close'] < stock_index['yes_upper']), -1, stock_index['signal'])

plt.subplot(2,1,1)
plt.title('沪深300 bolling交易信号图')
plt.gca().axes.get_xaxis().set_visible(False)
stock_index['close'].plot(figsize=(10,10))
plt.plot(stock_index['upper'], linestyle='--')
plt.plot(stock_index['middle'], linestyle='--')
plt.plot(stock_index['lower'], linestyle='--')
plt.legend()
plt.subplot(2,1,2)
plt.plot(stock_index['signal'], marker='o', linestyle='')
plt.legend()
plt.suptitle('沪深300 bolling交易信号图')
plt.show()

#使用循环法进行回测方法1
position = 0
for item in stock_index.iterrows(): #使用.iterrows()以特定格式循环提取行信息
    if item[1]['signal'] == 1: #如果signal为1
        position = 1 #signal为1，则position为1
    elif item[1]['signal'] == -1:
        position = -1 #signal为-1，则position为-1
    else:
        pass
    stock_index.loc[item[0], 'position'] = position #自动往下填充postion

#使用循环法进行回测方法2
position = 0
for i, item in stock_index.iterrows(): #使用.iterrows()以特定格式循环提取行信息
    if item['signal'] == 1:
        position = 1
    elif item['signal'] == -1:
        position = -1
    else:
        pass
    stock_index.loc[i, 'position'] = position #自动往下填充postion

# 绘制持仓情况图
plt.subplot(2,1,1)
plt.gca().axes.get_xaxis().set_visible(False)
stock_index['close'].plot(figsize=(10,8))
plt.plot(stock_index['upper'], linestyle='--')
plt.plot(stock_index['middle'], linestyle='--')
plt.plot(stock_index['lower'], linestyle='--')
plt.legend()
plt.subplot(2,1,2)
plt.plot(stock_index['position'], marker='o', linestyle='')
plt.legend()
plt.suptitle('沪深300 bolling持仓情况')

stock_index['pct_change'] = stock_index['close'].pct_change() #股票每日收益率
stock_index['return'] = (stock_index['pct_change'] + 1).cumprod() #股票累积收益率
stock_index['strategy_return'] = stock_index['position'] * stock_index['pct_change'] #策略每日收益率
stock_index['cum_strategy_return'] = (stock_index['strategy_return'] + 1).cumprod() #策略累积收益率

# 绘图
fig = plt.figure(figsize = (10,6))
plt.plot(stock_index['return'])
plt.plot(stock_index['cum_strategy_return'])
plt.title('沪深300 收益曲线图')
plt.legend(loc='upper left')
plt.show()